<a href="https://colab.research.google.com/github/dnguyend/ManNullRange/blob/master/colab/FlagLogarithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TESTING THE LOGARITHM MAP FOR FLAG MANIFOLDS

In [1]:
!git clone  https://github.com/pymanopt/pymanopt.git
import sys
sys.path.append("/content/pymanopt")


Cloning into 'pymanopt'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 4127 (delta 0), reused 0 (delta 0), pack-reused 4124
Receiving objects: 100% (4127/4127), 899.22 KiB | 1.07 MiB/s, done.
Resolving deltas: 100% (2876/2876), done.


In [2]:
!git clone https://github.com/dnguyend/ManNullRange.git

Cloning into 'ManNullRange'...
remote: Enumerating objects: 229, done.
remote: Counting objects: 100% (229/229), done.
remote: Compressing objects: 100% (170/170), done.
remote: Total 229 (delta 150), reused 100 (delta 58), pack-reused 0
Receiving objects: 100% (229/229), 680.17 KiB | 979.00 KiB/s, done.
Resolving deltas: 100% (150/150), done.


In [3]:
import numpy as np
from numpy.random import (randint, randn)
from numpy import zeros, zeros_like, trace, allclose

from ManNullRange.manifolds.RealFlag import RealFlag
from ManNullRange.tests.test_tools import (
    check_zero, make_sym_pos, random_orthogonal, timeout,
    TimeoutError)


def compare_with_grassmann(n, d, NN):
    import numpy.linalg as la
    from numpy import bmat
    from scipy.linalg import expm, expm_frechet
    from ManNullRange.manifolds.tools import (
        vecah, unvecah, asym)
    from pymanopt.manifolds import Grassmann
    
    np.random.seed(0)    
    alp = np.zeros(2)
    alp[0] = 1
    alp[1] = .8

    dvec = np.array([n-d, d])
    q = dvec.shape[0]-1
    alpha = randint(1, 10, (q, q+1)) * .1
    alpha[:, 0] = alp[0]
    alpha[:, 1:] = alp[1]
    m, d = dvec.sum(), dvec[1:].sum()
    man = RealFlag(
        dvec, alpha=alpha,
        log_stats=True, log_method='trust-krylov')

    gr = Grassmann(m, d)
    
    X = gr.rand()
    # xi = gr.randvec(X)
    # X1 = gr.exp(X, np.pi*xi)

    cnt = 0
    # NN = 1000
    for i in range(NN):
        xi = gr.randvec(X)
        X1 = gr.exp(X, 0.6*np.pi*xi)
        # X1 = gr.rand()
        v1 = gr.dist(X, X1)
        lgs, sts = man.log(X, X1, init_type=0)
        v2 = man.norm(X, lgs)
        if np.abs(v1 - v2) > 1e-4:
            print(i, v1, v2, v1-v2, (v1+v2)/np.pi)
            print(check_zero(X.T@lgs))
            X1a = man.exp(X, lgs)
            print(check_zero(X1a.T@X1a - np.eye(X1a.shape[1])))
            print(sts)
            # pdb.set_trace()
            cnt += 1
    print('Number of discrepency %d, percent discrepency %f' % (cnt, cnt/NN))
    
    print('Check that the aligment matrix is orthogonal')
    X1a = man.exp(X, lgs)
    F = X1.T@X1a
    print('L1 norm of diff is %f' % np.max(np.abs(F.T@F - np.eye(d))))

compare_with_grassmann(n=50, d=20, NN=100)

Number of discrepency 0, percent discrepency 0.000000
Check that the aligment matrix is orthogonal
L1 norm of diff is 0.000000


In [5]:

import numpy.linalg as la
from numpy import bmat
from scipy.linalg import expm, expm_frechet
from ManNullRange.manifolds.tools import (
    vecah, unvecah, asym)



# reporting statistics. Reporting the tables counting iteration, function valuations, improvements over generating vector

In [10]:
def make_lbd(dvec):
    dd = dvec.shape[0]

    def coef(a, dd):
        if dd % 2 == 0:
            if a < dd // 2 - 1:
                return - (dd // 2) + a + 1
            else:
                return - (dd // 2) + a + 2
        else:
            if a < (dd-1)//2:
                return -(dd-1)//2 + a
            else:
                return -(dd-1)//2 + a + 1

    dsum = dvec[1:].cumsum()
    lbd = np.concatenate([np.ones(dvec[a+1])*coef(a, dd)
                          for a in range(dsum.shape[0])])
    return lbd
    

def report_table(d_array):
    import numpy.linalg as la
    from numpy import bmat
    from scipy.linalg import expm, expm_frechet
    from ManNullRange.manifolds.tools import (
        vecah, unvecah, asym)
    
    alp = np.array([1, 1])
    
    dvec = np.array(d_array)
    # dvec = np.array([0, 20, 15, 5])
    # dvec = np.array([0, 10, 5, 5])    
    # dvec[0] = 1000 - dvec[1:].sum()
    # dvec = np.array([5, 3, 1])
    q = dvec.shape[0]-1
    alpha = randint(1, 10, (q, q+1)) * .1
    alpha[:, 0] = alp[0]
    alpha[:, 1:] = alp[1]
    m, d = dvec.sum(), dvec[1:].sum()
    man = RealFlag(
        dvec, alpha=alpha,
        log_stats=True, log_method='trust-krylov')

    lbd = make_lbd(dvec)
    
    def llbd(mat):
        return lbd[:, None]*mat

    def rlbd(mat):
        return mat*lbd[None, :]    
    
    NN = 50
    mean_sts = {}
    klist = ['nit', 'nfev', 'njev', 'nhev', 'success', 'dist', 'improve']
    fts = dict((kk, 1) for kk in klist)
    fts['success'] = 100
    np.random.seed(0)
    for radius in [.5, .99, 1.3]:
        rkey = 'rd%0.2f' % radius
        mean_sts[rkey] = {}
        for alx in np.array([1, 5, 10, 12]):
            tots = dict((k, {0: 0}) for k in klist)
            X = man.rand()        
            np.random.seed(0)            
            man.alpha[:, 1:] = alx*.1
            cnt = 0
            all_sts = {}
            itr = {0: 0}
            nhev = {0: 0}
            for i in range(NN):
                xi = man.randvec(X)
                X1 = man.exp(X, radius*np.pi*xi)
                try:
                    lgs0, sts0 = man.log(X, X1, init_type=0)
                except Exception:
                    sts0 = [('success', False)]
                    lgs0 = np.zeros_like(X)
                msg0 = dict(sts0)
                all_sts[i] = {0: sts0}
                v0 = man.norm(X, lgs0)
                X1a = man.exp(X, lgs0)
                diff = (lbd*lbd).sum() - np.trace(rlbd((X1.T@X1a))@rlbd((X1a.T@X1)))
                
                if diff > 1e-3:
                    msg0['success'] = False
                elif v0 <= radius*np.pi + 1e-3:
                    tots['dist'][0] += 1
                    tots['improve'][0] += 1-man.norm(X, lgs0)/radius/np.pi
                                        
                for kk in klist:
                    if kk in msg0:
                        tots[kk][0] += msg0[kk]
            mean_sts[rkey][alx] = dict((kk, [tots[kk][0]/NN*fts[kk]])
                                       for kk in klist)
            print('radius %s alpha %f' % (rkey, man.alpha[0, 1]))
    print(mean_sts)
    fname = 'tbl_flag_r_al_m%d_d%d.p' % (m, d)
    np.savez_compressed(fname, mean_sts)
    return mean_sts


In [8]:
mean_sts = report_table([960, 20, 15, 5])

radius rd0.50 alpha 0.100000
radius rd0.50 alpha 0.500000
radius rd0.50 alpha 1.000000
radius rd0.50 alpha 1.200000
radius rd0.99 alpha 0.100000
radius rd0.99 alpha 0.500000
radius rd0.99 alpha 1.000000
radius rd0.99 alpha 1.200000
radius rd1.30 alpha 0.100000
radius rd1.30 alpha 0.500000
radius rd1.30 alpha 1.000000
radius rd1.30 alpha 1.200000
{'rd0.50': {1: {'nit': [5.12], 'nfev': [6.12], 'njev': [6.12], 'nhev': [41.04], 'success': [100.0], 'dist': [1.0], 'improve': [1.023644802282675e-06]}, 5: {'nit': [5.0], 'nfev': [6.0], 'njev': [6.0], 'nhev': [21.0], 'success': [100.0], 'dist': [1.0], 'improve': [1.3187403339331993e-07]}, 10: {'nit': [5.0], 'nfev': [6.0], 'njev': [6.0], 'nhev': [23.82], 'success': [100.0], 'dist': [1.0], 'improve': [3.7526513696484184e-09]}, 12: {'nit': [5.0], 'nfev': [6.0], 'njev': [6.0], 'nhev': [27.28], 'success': [100.0], 'dist': [1.0], 'improve': [2.2096632279478e-09]}}, 'rd0.99': {1: {'nit': [6.0], 'nfev': [7.0], 'njev': [7.0], 'nhev': [29.92], 'success': 

In [11]:
mean_sts = report_table([5, 3, 1])

radius rd0.50 alpha 0.100000
radius rd0.50 alpha 0.500000
radius rd0.50 alpha 1.000000
radius rd0.50 alpha 1.200000
radius rd0.99 alpha 0.100000
radius rd0.99 alpha 0.500000
radius rd0.99 alpha 1.000000
radius rd0.99 alpha 1.200000
radius rd1.30 alpha 0.100000
radius rd1.30 alpha 0.500000
radius rd1.30 alpha 1.000000
radius rd1.30 alpha 1.200000
{'rd0.50': {1: {'nit': [5.9], 'nfev': [6.9], 'njev': [6.9], 'nhev': [29.16], 'success': [100.0], 'dist': [1.0], 'improve': [0.017887709223152935]}, 5: {'nit': [5.54], 'nfev': [6.54], 'njev': [6.54], 'nhev': [22.66], 'success': [100.0], 'dist': [1.0], 'improve': [-2.265965755097188e-06]}, 10: {'nit': [5.42], 'nfev': [6.42], 'njev': [6.42], 'nhev': [19.48], 'success': [100.0], 'dist': [1.0], 'improve': [-1.0322560332642894e-06]}, 12: {'nit': [5.48], 'nfev': [6.48], 'njev': [6.48], 'nhev': [21.86], 'success': [100.0], 'dist': [1.0], 'improve': [-8.579752165216448e-07]}}, 'rd0.99': {1: {'nit': [9.98], 'nfev': [10.94], 'njev': [10.94], 'nhev': [73.4

In [12]:
def print_tbl(mean_sts):
    header = [r'dist ($\pi$)', r'$\alpha$', r'\#Iteration', r'\#Evals',
              r'\%Succ', '\\%Improve', '\\%Not worse']
    radius = sorted(mean_sts)

    def print_block():
        for r in radius:
            for alf in sorted(mean_sts[r].keys()):
                ddc = mean_sts[r][alf]
                out_str = '%s & %0.2f & %0.2f & %.2f &  %.f & %.f & %.f\\\\'
                print(
                    out_str % (
                        r[2:], alf / 10, ddc['nit'][0],
                        ddc['nfev'][0] + 4*ddc['nhev'][0] + 2*ddc['njev'][0],
                        ddc['success'][0],
                        np.abs(ddc['improve'][0]*100),
                        ddc['dist'][0]*100))
    print('&'.join(header) + r'\\')
    print_block()


def print_all():
    m1000_40 = np.load('tbl_flag_r_al_m1000_d40.p.npz', allow_pickle=True)['arr_0'].tolist()
    print_tbl(m1000_40)

    m4_2 = np.load('tbl_flag_r_al_m9_d4.p.npz', allow_pickle=True)['arr_0'].tolist()
    print_tbl(m4_2)

print_all()

dist ($\pi$)&$\alpha$&\#Iteration&\#Evals&\%Succ&\%Improve&\%Not worse\\
0.50 & 0.10 & 5.12 & 182.52 &  100 & 0 & 100\\
0.50 & 0.50 & 5.00 & 102.00 &  100 & 0 & 100\\
0.50 & 1.00 & 5.00 & 113.28 &  100 & 0 & 100\\
0.50 & 1.20 & 5.00 & 127.12 &  100 & 0 & 100\\
0.99 & 0.10 & 6.00 & 140.68 &  100 & 0 & 100\\
0.99 & 0.50 & 5.00 & 106.00 &  100 & 0 & 100\\
0.99 & 1.00 & 5.00 & 96.16 &  100 & 0 & 100\\
0.99 & 1.20 & 5.00 & 98.00 &  100 & 0 & 100\\
1.30 & 0.10 & 7.00 & 164.00 &  100 & 0 & 100\\
1.30 & 0.50 & 6.70 & 295.10 &  100 & 0 & 100\\
1.30 & 1.00 & 6.00 & 131.64 &  100 & 0 & 100\\
1.30 & 1.20 & 6.00 & 127.96 &  100 & 0 & 100\\
dist ($\pi$)&$\alpha$&\#Iteration&\#Evals&\%Succ&\%Improve&\%Not worse\\
0.50 & 0.10 & 5.90 & 137.34 &  100 & 2 & 100\\
0.50 & 0.50 & 5.54 & 110.26 &  100 & 0 & 100\\
0.50 & 1.00 & 5.42 & 97.18 &  100 & 0 & 100\\
0.50 & 1.20 & 5.48 & 106.88 &  100 & 0 & 100\\
0.99 & 0.10 & 9.98 & 326.66 &  96 & 32 & 96\\
0.99 & 0.50 & 9.18 & 271.98 &  100 & 28 & 100\\
0.99 & 1.00

# Computing Riemannian center of Mass

In [46]:
def riemannian_center_of_mass_loop():
    # now do Riemannian center of mass
    import time    
    np.random.seed(0)
    radius = .5
    
    dvec = np.array([0, 30, 15, 5])
    # dvec = np.array([0, 10, 5, 5])
    
    dvec[0] = 1000 - dvec[1:].sum()
    p = dvec.shape[0]-1
    alpha = randint(1, 10, (p, p+1)) * .1
    alpha[:, 0] = 1
    alpha[:, 0] = 1.
    
    N = 50
    m, d = dvec.sum(), dvec[1:].sum()
    man = RealFlag(
        dvec, alpha=alpha, log_stats=False, log_method='trust-krylov')
    # man.log_gtol = 1e-6

    Q = np.zeros((m, d, N))
    Y = man.rand()
    fts = np.zeros(N)

    for i in range(N):
        fts[i] = np.random.uniform(radius*.8, radius)*np.pi
        Q[:, :, i] = man.exp(Y, fts[i]*man.randvec(Y))

    man.log_stats = True
    tot_stats_all = {}
    init_type = 0
    for alx in [1, 5, 10]:
        start_time = time.time()        
        man.alpha[:, 1:] = alx*.1
        man.log_stats = True
        if alx != 10:
            ctr, tot_stats, itr_stats = RCM(man, Q, 0, init_type=init_type)
        else:
            ctr, tot_stats, itr_stats = RCM(man, Q, 0, init_type=init_type)
        print('m=%d d=%d alpha_1=%f radius=%f dist_to_y=%f' %
              (*ctr.shape, man.alpha[0, 1], radius,
               man.norm(Y, man.log(Y, ctr)[0])))
        for itr in sorted(itr_stats.keys()):
            print('itr=%d stat=%s' % (itr, itr_stats[itr]))
        print(tot_stats)
        tot_stats_all[alx] = (tot_stats, itr_stats)
        end_time = time.time()
        print(start_time, end_time, end_time - start_time)

    import pickle
    with open('rcm_flag.p', 'wb') as ff:
        pickle.dump(tot_stats_all, ff)
    return tot_stats_all
        

def RCM(man, Q, init_idx=None, init_type=0):
    m, d = man.dvec.sum(), man.dvec[1:].sum()
    N = Q.shape[2]
    n_iter = 100
    a, _, b = np.linalg.svd(0.8*Q[:, :, 0] + 0.2*Q.mean(axis=2), full_matrices=False)
    if init_idx is None:
        Yi = a@b
    else:
        Yi = Q[:, :, init_idx]
    Yinew = Yi.copy()
    k_list = ['success', 'nfev', 'njev', 'nhev', 'nit']
    tot_stats = dict((kk, 0) for kk in k_list)
    tot_stats['cnt'] = 0
    itr_mean = {}
    tot_mean = {}

    for itr in range(n_iter):
        logs = np.zeros((m, d))
        itr_stats = dict((kk, 0) for kk in k_list)
        itr_stats['cnt'] = 0
        itr_mean[itr] = {}
        variance = 0        
        for i in range(N):
            itr_stats['cnt'] += 1
            tot_stats['cnt'] += 1
            if not man.log_stats:
                ret = man.log(Yi, Q[:, :, i])
            else:
                ret, sts = man.log(Yi, Q[:, :, i], init_type=init_type)
                msg = dict(sts)
                for kk in k_list:
                    if kk in msg:
                        itr_stats[kk] += msg[kk]
                        tot_stats[kk] += msg[kk]                        

                if False and (msg['success'] is not True):
                    print('itr = %d i=%d %s' % (itr, i, str(msg)))
            jlogs = man.J(Yi, ret)
            if np.isnan(ret).sum() == 0:
                logs += 1/N*ret
                variance += 1/N*man.inner(Yi, ret, ret)
        nlg = man.norm(Yi, logs)
        for kk in k_list:
            if kk == 'success':
                itr_mean[itr][kk] = itr_stats[kk] / itr_stats['cnt']
            else:
                itr_mean[itr][kk] = itr_stats[kk] / itr_stats['success']

        # sometime the optimizer return sucess = False
        # but opt value is still good to use
        # so count of success does not mean that much
        # iteration get much slower
        itr_mean[itr]['success'] *= 100
        itr_mean[itr]['norm_RCM_gradient'] = nlg
        itr_mean[itr]['RCM_variance'] = variance
        # print('itr=%d dis=%f %s' % (itr, nlg, str(itr_mean)))
        if nlg < 1e-5:
            break
        Yinew = man.exp(Yi, logs)
        Yi = Yinew
        for kk in k_list:
            if kk == 'success':
                tot_mean[kk] = (tot_stats[kk] / tot_stats['cnt'])
            else:
                tot_mean[kk] = (tot_stats[kk] / tot_stats['success'])
        tot_mean['success'] *= 100

    return Yi, tot_mean, itr_mean

tot_stats_all = riemannian_center_of_mass_loop()
print(tot_stats_all)

m=1000 d=50 alpha_1=0.100000 radius=0.500000 dist_to_y=0.204099
itr=0 stat={'success': 100.0, 'nfev': 6.28, 'njev': 6.28, 'nhev': 24.64, 'nit': 5.28, 'norm_RCM_gradient': 1.3602580701907503, 'RCM_variance': 3.8651814622567935}
itr=1 stat={'success': 100.0, 'nfev': 5.98, 'njev': 5.98, 'nhev': 21.0, 'nit': 4.98, 'norm_RCM_gradient': 0.018713896541292636, 'RCM_variance': 1.9894232532351745}
itr=2 stat={'success': 100.0, 'nfev': 5.98, 'njev': 5.98, 'nhev': 21.02, 'nit': 4.98, 'norm_RCM_gradient': 0.00025882138506559145, 'RCM_variance': 1.9890682933007375}
itr=3 stat={'success': 100.0, 'nfev': 5.98, 'njev': 5.98, 'nhev': 21.02, 'nit': 4.98, 'norm_RCM_gradient': 3.624879148056597e-06, 'RCM_variance': 1.9890682170692502}
{'success': 100.0, 'nfev': 6.08, 'njev': 6.08, 'nhev': 22.22, 'nit': 5.08}
1609554498.2238028 1609554617.8652682 119.64146542549133
m=1000 d=50 alpha_1=0.500000 radius=0.500000 dist_to_y=0.204668
itr=0 stat={'success': 100.0, 'nfev': 5.94, 'njev': 5.94, 'nhev': 18.32, 'nit': 

{1: ({'nfev': 6.08,
   'nhev': 22.22,
   'nit': 5.08,
   'njev': 6.08,
   'success': 100.0},
  {0: {'RCM_variance': 3.8651814622567935,
    'nfev': 6.28,
    'nhev': 24.64,
    'nit': 5.28,
    'njev': 6.28,
    'norm_RCM_gradient': 1.3602580701907503,
    'success': 100.0},
   1: {'RCM_variance': 1.9894232532351745,
    'nfev': 5.98,
    'nhev': 21.0,
    'nit': 4.98,
    'njev': 5.98,
    'norm_RCM_gradient': 0.018713896541292636,
    'success': 100.0},
   2: {'RCM_variance': 1.9890682933007375,
    'nfev': 5.98,
    'nhev': 21.02,
    'nit': 4.98,
    'njev': 5.98,
    'norm_RCM_gradient': 0.00025882138506559145,
    'success': 100.0},
   3: {'RCM_variance': 1.9890682170692502,
    'nfev': 5.98,
    'nhev': 21.02,
    'nit': 4.98,
    'njev': 5.98,
    'norm_RCM_gradient': 3.624879148056597e-06,
    'success': 100.0}}),
 5: ({'nfev': 5.98,
   'nhev': 20.013333333333332,
   'nit': 4.98,
   'njev': 5.98,
   'success': 100.0},
  {0: {'RCM_variance': 3.88863586332686,
    'nfev': 5.94,


# Print Hessian Spectra

In [48]:
def test_hessian_at_zero_A_R():
    import numpy.linalg as la
    from numpy import bmat
    from scipy.linalg import expm, expm_frechet
    from ManNullRange.manifolds.tools import (
        vecah, unvecah, asym)
    np.random.seed(0)    
    alp = np.zeros(2)
    alp[0] = 1
    alp[1] = .8

    dvec = np.array([0, 30, 15, 5])
    dvec[0] = 1000 - dvec[1:].sum()    
    dvec = np.array([15, 3, 2, 2])

    def make_lbd():
        dd = dvec.shape[0]

        def coef(a, dd):
            if dd % 2 == 0:
                if a < dd // 2 - 1:
                    return - (dd // 2) + a + 1
                else:
                    return - (dd // 2) + a + 2
            else:
                if a < (dd-1)//2:
                    return -(dd-1)//2 + a
                else:
                    return -(dd-1)//2 + a + 1
                
        dsum = dvec[1:].cumsum()
        lbd = np.concatenate([np.ones(dvec[a+1])*coef(a, dd)
                              for a in range(dsum.shape[0])])
        # return .5 + lbd / lbd.sum()
        return lbd
    
    lbd = make_lbd()
    
    p = dvec.shape[0]-1
    alpha = randint(1, 10, (p, p+1)) * .1
    alpha[:, 0] = alp[0]
    alpha[:, 1:] = alp[1]
    sqrt2 = np.sqrt(2)
    N = 50
    m, d = dvec.sum(), dvec[1:].sum()
    man = RealFlag(
        dvec, alpha=alpha, log_stats=False, log_method='trust-krylov')
    
    def vec(A, R):
        # for A, take all blocks [ij with i > j]
        lret = []
        for r in range(1, p+1):
            gdc = man._g_idx                
            if r not in gdc:
                continue
            br, er = gdc[r]
            for s in range(r+1, p+1):
                if s <= r:
                    continue
                bs, es = gdc[s]
                lret.append(A[br:er, bs:es].reshape(-1)*sqrt2)

        lret.append(R.reshape(-1))
        return np.concatenate(lret)

    def unvec(avec):
        A = np.zeros((d, d))
        R = np.zeros((k, d))
        gdc = man._g_idx
        be = 0
        for r in range(1, p+1):
            if r not in gdc:
                continue
            br, er = gdc[r]
            for s in range(r+1, p+1):
                if s <= r:
                    continue
                bs, es = gdc[s]
                dr = er - br
                ds = es - bs
                A[br:er, bs:es] = (avec[be: be+dr*ds]/sqrt2).reshape(dr, ds)
                A[bs:es, br:er] = - A[br:er, bs:es].T
                be += dr*ds
        R = avec[be:].reshape(k, d)
        return A, R

    def asym(mat):
        return 0.5*(mat - mat.T)

    def make_vec(xi):
        return vec(X.T@xi, Q.T@xi)

    def conv_to_tan(A, R):
        return X@A + Q@R
    
    def calc_hessian_spec(lgs, X1):        
        XQ = np.array(np.bmat([X, Q]))
        X2 = XQ.T@(X1*lbd[None, :])@X1.T@XQ
                
        def dist(v):
            #  = (dist0a(v) - d)*2
            alf = man.alpha[0, 1] / man.alpha[0, 0]
            A, R = unvec(v)
            x_mat = np.array(
                np.bmat([[2*alf*A, -R.T], [R, zeros((k, k))]]))
            exh = expm(x_mat)
            ex = expm((1-2*alf)*A)
            Mid = (ex*lbd[None, :])@ex.T
            return (- trace(X2@exh[:, :d]@Mid@exh[:, :d].T))

        def jac(v):
            alf = man.alpha[0, 1] / man.alpha[0, 0]            
            gdc = man._g_idx
            A, R = unvec(v)
            x_mat = np.array(
                np.bmat([[2*alf*A, -R.T], [R, zeros((k, k))]]))
            exh = expm(x_mat)
            ex = expm((1-2*alf)*A)

            blk = np.zeros_like(exh)
            blk[:d, :] = (ex*lbd[None, :])@ex.T@exh[:, :d].T
            blkA = (lbd[:, None]*ex.T)@exh[:, :d].T@X2@exh[:, :d]

            fexh = 2*expm_frechet(x_mat, blk@X2)[1]
            fex = 2*expm_frechet((1-2*alf)*A, blkA)[1]

            for r in range(1, p+1):
                if r not in gdc:
                    continue
                br, er = gdc[r]            
                fexh[br:br, br:br] = 0
                fex[br:br, br:br] = 0

            return vec(
                (1-2*alf)*asym(fex) + 2*alf*asym(fexh[:d, :d]),
                fexh[d:, :d] - fexh[:d, d:].T)
        
        def hessp(v, xi):
            dlt = 1e-8
            return (jac(v+dlt*xi) - jac(v))/dlt
        
        mat = np.zeros((tdim, tdim))
        Alg = X.T@lgs
        Rlgs = Q.T@lgs
        vlgs = vec(Alg, Rlgs)
        for i in range(tdim):
            av = np.zeros(tdim)
            av[i] = 1
            mat[:, i] = hessp(vlgs, av)
        mat = 0.5*(mat + mat.T)
        evl, evec = la.eigh(mat)
        return evl

    k = min(d, m-d)
    adim = (dvec[1:].sum()*dvec[1:].sum() - (dvec[1:]*dvec[1:]).sum()) // 2
    tdim = d*k + adim

    X = man.rand()
    eta0 = man.randvec(X)
    Q, _ = la.qr(eta0 - X@X.T@eta0)

    X1 = X.copy()
    # X1 = man.exp(X, 1*eta0)
    
    v = np.zeros(tdim)
    
    # A0, R0 = unvec(np.zeros((tdim)))
    # Q = make_Q(k)
    spt = calc_hessian_spec(np.zeros((m, d)), X1)
    print("Eigenvalues of the Hessian at 0")
    print(spt)

    from scipy.optimize import minimize_scalar
    man.alpha[:, 1:] = .01-.1
    cut_val = {}

    for aj in range(12):
        np.random.seed(0)
        man.alpha[:, 1:] += .1
        NN = 50
        cut_val[aj] = {}
        for ii in range(NN):
            xi = man.randvec(X)

            def make_Q(k):
                Q, _ = la.qr(xi - X@X.T@xi)
                return Q
            Q = make_Q(d)

            def min_spec(t):
                Y1 = man.exp(X, t*xi)
                spt = calc_hessian_spec(t*xi, Y1)
                # print(spt)
                return spt[0]*spt[0]

            ret = minimize_scalar(
                min_spec, method='bounded', bounds=(.01*np.pi, 1.4*np.pi))
            if ret['success'] and ret['fun'] < 1e-3:
                cut_val[aj][ii] = (ret['x'], ret['fun'])
            else:
                print(ret)
        print('quasi cut values for alpha=%f' % man.alpha[0, 1])
        print([cut_val[aj][a] for a in
               sorted(cut_val[aj], key=lambda i: cut_val[aj][i][0])])
    print("spetra - changing with t")
    if True:        
      for i in range(100):
          t = np.pi*1.4*i/100
          X1 = man.exp(X, t*xi)
          spt = sorted(calc_hessian_spec(t*xi, X1))
          print(t, spt[0], spt[-1])

test_hessian_at_zero_A_R()

Eigenvalues of the Hessian at 0
[1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 4. 4. 4. 4. 4. 4. 8. 8. 8.
 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 9. 9. 9. 9. 9. 9.]
quasi cut values for alpha=0.010000
[(2.77244173235919, 9.292963356931743e-17), (2.917478364883905, 1.1513702517563276e-17), (3.0024235562658546, 4.573012376441107e-16), (3.010956969086629, 1.7577155287451728e-18), (3.0287907168850188, 5.754806808639748e-18), (3.0297049757288463, 3.5312046657741235e-16), (3.031364355456592, 5.708802728310529e-20), (3.0361047497353644, 7.007030339777258e-18), (3.0424259165983627, 1.5805479993715925e-19), (3.046051541773083, 2.457163419779365e-19), (3.050970630804617, 4.41129265004565e-18), (3.0511793507012794, 1.357127824173867e-08), (3.051930599232298, 2.026044444789832e-16), (3.0598910190817437, 1.1039827168709643e-18), (3.0639838025564874, 2.609773330682844e-18), (3.072430160257471, 1.0468175926540441e-17), (3.0761801397133